<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/python/01RAD2023_assignment_HW_01_python_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
df61 = pd.read_csv("https://raw.githubusercontent.com/francji1/01RAD/main/data/sarasota_houses_mod.csv", sep=",")
df61.head()

#Question 01:

Find out if the data contains any missing values NA. If so, decide whether you can remove the relevant observations from the data and why. Which variables are quantitative and which are qualitative? If some can be classified into both groups, which one would you choose? Which variables can be used as ordinal factors and which as nominal factors and why? Calculate basic statistics for individual variables.



Solution 01:

In [ ]:
# Check for missing values
missing_values = df61.isnull().sum()

# Determine variable types
quantitative_variables = df61.select_dtypes(include=['int64', 'float64']).columns.tolist()
qualitative_variables = df61.select_dtypes(include=['object', 'bool']).columns.tolist()

# Basic statistics for quantitative variables
basic_stats = df61.describe()

# Outputs
print("Missing values in each column:\n", missing_values)
print("\nQuantitative variables:", quantitative_variables)
print("\nQualitative variables:", qualitative_variables)
print("\nBasic statistics for quantitative variables:\n", basic_stats)

In [ ]:
# Check for NA values and remove rows with NA values
df61 = df61.dropna()

Question 02

We want to buy a property abroad and by surveying the market we have received the previous data set saratoga_houses. However, since we have more precise requirements and do not understand imperial units, we need to adjust the data:

Convert the property prices from dollars to Czech crowns in thousands and the land area and living area from acres and square feet to square meters. (saratoga_houses_mod)
Select only properties older than 10 years and younger than 50 years, whose price is less than 7,500,000 CZK, and the land area is between 500 and 5000 square meters.
Convert the number of bathrooms and the number of rooms into three-level factorial variables.
Continue working only with this limited dataset and with variables price, living_area, land_area, number_of_rooms, age_of_house, number_of_bathrooms, fireplace.
###Use:

1 USD is approximately 22 CZK

1 acre is approximately 4046.86 square meters.

1 square foot is approximately 0.092903 square meters.



Solution 02:

In [ ]:
# Conversion rates
usd_to_czk = 22  # Example conversion rate
acre_to_sqm = 4046.86
sqft_to_sqm = 0.092903

# Convert prices from USD to thousands of CZK
df61['price_czk_thousands'] = df61['price'] * usd_to_czk / 1000

# Convert lot size from acres to square meters
df61['lot_size_sqm'] = df61['lot_size'] * acre_to_sqm

# Convert living area from square feet to square meters
df61['living_area_sqm'] = df61['living_area'] * sqft_to_sqm

# Filter the dataframe
df_filtered = df61[
    (df61['age'] > 10) & (df61['age'] < 50) &
    (df61['price_czk_thousands'] < 7500) &
    (df61['lot_size_sqm'].between(500, 5000))
]


# Convert number of bathrooms and bedrooms into categorical variables with 3 levels
# For simplicity, let's define the levels as follows: 1, 1.5-2, >2 for bathrooms and 1-2, 3, >3 for bedrooms
df_filtered['bathrooms_factor'] = pd.cut(df_filtered['bathrooms'],
                                         bins=[0, 1, 2, float('inf')],
                                         labels=['1', '1.5-2', '>2'],
                                         right=False)
df_filtered['bedrooms_factor'] = pd.cut(df_filtered['bedrooms'],
                                        bins=[0, 2, 3, float('inf')],
                                        labels=['1-2', '3', '>3'],
                                        right=False)

# Select only the relevant columns for further analysis
df_final = df_filtered[
    ['price_czk_thousands', 'living_area_sqm', 'lot_size_sqm',
     'bedrooms_factor', 'age', 'bathrooms_factor', 'fireplace']
]

df_final.head()




Question 03

Compare the average prices of properties with a fireplace and without a fireplace and test whether at the significance level
�
=
0.01
α=0.01 the average price of properties with a fireplace is greater than the average price of properties without a fireplace.
Solution 03:


In [ ]:
from scipy import stats

# Calculate the average price for properties with and without a fireplace
avg_price_with_fireplace = df61[df61['fireplace'] == True]['price'].mean()
avg_price_without_fireplace = df61[df61['fireplace'] == False]['price'].mean()

# Perform an independent samples t-test
t_stat, p_value = stats.ttest_ind(
    df61[df61['fireplace'] == True]['price'].dropna(),
    df61[df61['fireplace'] == False]['price'].dropna(),
    equal_var=False  # Assume variances are not equal
)

avg_price_with_fireplace, avg_price_without_fireplace, p_value, p_value < 0.01



## Data Visualization

Question 04



Plot scatter plots for all numerical variables, with a color distinction whether it is a property with or without a fireplace.
For variables number_of_rooms, number_of_floors and basement, plot box plots (or violin plots), where the response will be price.
For the variable price, plot a histogram along with a kernel density estimate.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatter plots for all numerical variables
numerical_vars = ['price', 'living_area', 'bathrooms', 'bedrooms', 'fireplaces', 'lot_size', 'age']
for var in numerical_vars:
    sns.scatterplot(data=df61, x=var, y='price', hue='fireplace')
    plt.title(f'Scatter plot of Price vs {var}')
    plt.show()

# Box plots for 'number_of_rooms' and 'number_of_bathrooms' with 'price' as the response variable
# Since we don't have 'number_of_floors' and 'basement' in the dataset, we'll skip those.
sns.boxplot(data=df61, x='bedrooms', y='price')
plt.title('Box plot of Price by Number of Bedrooms')
plt.show()

sns.boxplot(data=df61, x='bathrooms', y='price')
plt.title('Box plot of Price by Number of Bathrooms')
plt.show()

# Histogram for the 'price' variable with a kernel density estimate
sns.histplot(df61['price'], kde=True)
plt.title('Histogram of Property Prices with Kernel Density Estimate')
plt.show()


In [ ]:
df_final

In [ ]:
df_final.dtypes


Question 05

For combinations of factorized variables number_of_rooms, number_of_bathrooms, plot the property price so that the image shows whether the average prices of properties with more rooms or more bathrooms differ, and whether all combinations of all levels for the two mentioned factorial variables are represented.



Question 06

For properties with two bedrooms, plot the dependence of the price on the living area of the property, where individual events will be marked with a color according to whether they have a fireplace, and the size of the points in the graph will correspond to the number of bathrooms (for this task it is better to take the number of bathrooms as a numerical variable).

Continue working only with properties with two bedrooms.



## Simple Linear Model

Question 07

Build a simple regression model (with and without an intercept), where the explained variable will be the property price and the explanatory living area. Calculate $R^2$ and $F$ statistics for both models, what do they tell us about the models. Choose one of them and justify why you prefer it.

Based on the chosen model, answer whether the property price depends on the living area and if so, by how much will the expected price change for a property with a living area increased by
$20 m^2$?



Question 08

Build a simple linear model as in the previous question for properties with and without a fireplace. Which model shows a stronger linear relationship between price and living area? By how much does the price with increasing living area for properties with a fireplace increase faster than for properties without a fireplace?

Calculate 95% confidence intervals for the regression coefficients describing the slope of the regression line in both models and find out whether they intersect. What can we deduce from that?

Based on these models, find out by what percentage the average property with a fireplace and a living area of
160
�
2
160m
2
  will have a higher expected price than the average property of the same living area, but without a fireplace.



Question 09

Plot a scatter plot of living area and property prices. In this graph, plot the regression lines of selected models for properties with and without a fireplace, mark the individual points and regression lines with a color according to which group they belong.

Construct 90% confidence intervals around the expected prices for individual groups and based on them, decide whether and how the expected prices will differ for properties with a living area less than $120m^2$. Is this comparison correct? Justify.



Question 10

Plot histograms for the residuals of the models from the previous question. Fit them with the density of a normal distribution with a zero mean and variance corresponding to $\hat{\sigma}^2$ from the individual models.

What does the resulting graph say about our models and is this verification sufficient to validate the model?

Suggest further modifications to the model in order to predict the property price as accurately as possible.